In [25]:
import os
import traceback
from pprint import pprint
from perspective import Table, PerspectiveWidget
from timeplus import Stream, Query, Environment

api_key = "apikey"
api_address = "https://dev.timeplus.cloud"
workspace = "tp-demo"

# Configure API key and address
env = Environment().address(api_address).workspace(workspace).apikey(api_key)

In [26]:
import json

sql = '''
SELECT 
  *
FROM 
  car_live_data
'''

query = (
        Query(env=env).sql(query=sql)
        .create()
    )

query_header = query.header()

In [27]:
print(query_header)

[{'name': 'in_use', 'type': 'bool'}, {'name': 'longitude', 'type': 'float32'}, {'name': 'latitude', 'type': 'float32'}, {'name': 'speed_kmh', 'type': 'uint32'}, {'name': 'gas_percent', 'type': 'decimal(10, 2)'}, {'name': 'total_km', 'type': 'float32'}, {'name': 'locked', 'type': 'bool'}, {'name': 'cid', 'type': 'string'}, {'name': 'time', 'type': 'datetime64(3)'}, {'name': '_tp_time', 'type': 'datetime64(3)'}]


In [28]:
from perspective import Table, PerspectiveWidget
import json
import pandas as pd

columns = [ f['name'] for f in query_header]

first_row = next(query.result())
if first_row.event == "message":
    data = json.loads(first_row.data)
    df = pd.DataFrame(data, columns=columns)
    
table = Table(df, index="cid")
PerspectiveWidget(table)

PerspectiveWidget(columns=['index', 'in_use', 'longitude', 'latitude', 'speed_kmh', 'gas_percent', 'total_km',…

In [ ]:
for event in query.result():
    if event.event == "message":
        new_data = json.loads(event.data)
        update_data = {}
        for col in columns:
            update_data[col] = []
        for row in new_data:
            for index, col in enumerate(columns):
                update_data[col].append(row[index])   
        table.update(update_data)